In [ ]:
epochs = 10
n_test_batches = 200

# Teil 11 - Sichere Deep Learning Klassifikation 



## Die Daten sind von Bedeutung, das Model ebenfalls

Die Daten treiben das Machine Learning voran. Organisationen welche Daten erschaffen und sammeln sind in der Lage selbst Machine Learning Modele zu trainieren. Dies ermöglicht ihnen ihre Modele als Service (MLaaS) anderen Organisationen bereit zu stellen. Dies ist insbesondere nützlich für Organisationen, die nicht genug Daten für ein eigenes Model besitzen, aber dennoch von Vorhersagen solch eines Models profitieren möchten. 

Doch so ein bereit gestelltes Model in der Cloud stellt immer noch ein Problem für die Privatsphäre dar. Um von anderen Organisationen genutzt werden zu können, müssen entweder die Input-Daten (wie z. B. zu klassifizierende Bilder) hochgeladen oder das gesamte Model herunter geladen werden. Das Hochladen der Daten kann problematisch sein aus Sicht der zu schützenden Privatsphäre. Das Herunterladen des Models ist jedoch nicht immer eine Option, wenn zum Beispiel die Organisation dadurch ihr geistiges Eigentum verliert.

## Berechnungen mit verschlüsselten Daten

In diesem Kontext kann eine mögliche Lösung im Verschlüsseln der Daten und des Models liegen. So können nach dem Verschlüsseln verschiedene Organisationen zusammen arbeiten, ohne gleich ihr IP preiszugeben. Es existieren mehrere Verschlüsselungs Schemata, die es ermöglichen mit verschlüsselten Daten zu rechnen. Zu den Bekanntesten gehören "Secure Multi-Party Computation" (SMPC), "Homomorphic Encryption" (FHE/SHE) und "Functional Encryption" (FE). Hier wird der Fokus auf "Secure Multi-Party Computation" mit dem additiven Aufteilen aus [Tutorial 5](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%205%20-%20Intro%20to%20Encrypted%20Programs.ipynb) gelegt. Dabei wird auf den Crypto-Protokollen SecureNN und SPDZ ([Blogpost](https://mortendahl.github.io/2017/09/19/private-image-analysis-with-mpc/)) aufgebaut. 

Diese Protokolle zeigen außergewöhnliche Performance auf verschlüsselten Daten und in den vergangenen Monaten wurde daran gearbeitet sie einfacher nutzen zu können. Im Speziellen wurde an Werkzeugen gearbeitet, die eine Nutzung der Protokolle ermöglichen, ohne sie selbst implementieren zu müssen oder gar ein tieferes Verständnis für die zugrundeliegenden Strukturen zu besitzen.

## Setup

Der genaue Hintergrund dieses Tutorials ist wie folgt: es gibt einen Server mit einigen Daten. Zuerst wird ein Model auf diesen geheimen Daten lokal trainiert. Anschließend wird Kontakt zu einem Klienten aufgenommen, welcher ebenfalls Daten sein Eigen nennt und das erstellte Model für seine Vorhersagen nutzen möchte. 

Das Model (ein neuronales Netzwerk) wird deshalb verschlüsselt. Der Klient verschlüsselt seine eigenen Daten. Daraufhin nutzen beide Beteiligte die aufgeteilten Anteile der Daten und des Models, um die verschlüsselten Daten zu klassifizieren. Schließlich wird das verschlüsselte Ergebnis zurück an den Klienten gesendet. Somit kann der Server nichts über die Daten oder deren Vorhersage schlussfolgern. 

Optimaler Weise müssten die Eingaben des `client` additiv aufgeteilt werden zwischen sich und dem `server`. Umgekehrt gilt dies genauso für das Model. Wegen der Übersichtlichkeit wird dieser Prozess hier auf die beiden Helfer `alice` und `bob` abgebildet. Wird angenommen Alice sei der Klient und Bob der Server, so ist das Beispiel equivalent.

Diese Berechnungen sind sicher gegen einen "ehrlichen-aber-neugierigen" Angriff, welcher Standard in [vielen MPC Gerüsten](https://arxiv.org/pdf/1801.03239.pdf) ist.

**Nun ist alles bereit und es kann begonnen werden!**

Autoren:
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel) · GitHub: [@LaRiffle](https://github.com/LaRiffle)

Übersetzer:
- Jan Moritz Behnken - Github: [@JMBehnken](https://github.com/JMBehnken)

### Importe und Model Spezifikationen

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Auch müssen Importe speziell für PySyft getätigt werden. Einige Helfer (genannt `client`, `bob` und `alice`) werden erstellt. Zuletzt wird der `crypto_provider` erzeugt, welcher die sichere Verschlüsselung ermöglicht ([genaueres in diesem Tutorial](https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%2009%20-%20Intro%20to%20Encrypted%20Programs.ipynb)).

In [ ]:
import syft as sy
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider") 

Die Einstellungen für das Training werden festgelegt.

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = epochs
        self.lr = 0.001
        self.log_interval = 100

args = Arguments()

### Laden der Daten und versenden an die Helfer

Zu den Grundannnahmen zählt, dass der Server Zugang zu Daten hat um ein Model zu trainieren. In diesem Fall ist es der MNIST Trainings-Datensatz.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

Weiterhin wird angenommen, dass der Klient eigene Daten besitzt und diese mit dem Model des Servers nutzen möchte. Dieser Klient verschlüsselt seine Daten indem er sie additiv auf die beiden Helfer `alice` und `bob` aufteilt. 

> SMPC verwendet Crypto-Protokolle, welche Integer voraussetzen. Dafür wird die PySyft Funktion `.fix_precision()` verwendet, welche Float-Tensoren dementsprechend anpasst. Beispielsweise wird 0.123 mit einer Genauigkeit von 2 auf die zweite Nachkommastelle gerundet und dann als Integer 12 gespeichert. 

In [ ]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_precision().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
    ))

### Spezifikationen eines vorwärts gerichteten Neuronalen Netzwerkes

Dies ist das Netzwerk vom Server.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

### Starten des Trainings

Das Training findet lokal auf der Maschine statt und wurde in reinem PyTorch umgesetzt.

In [ ]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)


In [ ]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            output = F.log_softmax(output, dim=1)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
test(args, model, test_loader)

Das Model ist nun trainiert und bereit als Service angeboten zu werden!

## Sicheres Evaluieren

Der Server verschlüsselt nun sein Model und versendet es an die Helfer. Weil das Model vertrauliche Informationen enthält (schließlich wurde Zeit und Geld in dessen Optimierung investiert), werden alle Matrizen des Models mittels additivem Aufteilen vor dem Versenden verschlüsselt.

In [ ]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

Diese Test Funktion berechnet eine verschlüsselte Evaluation. Die Gewichte des Models, die Eingabe-Daten, die Ausgabe-Daten und auch die Ziel-Variable sind dabei verschlüsselt!

Trotzdem bleibt die Syntax sehr ähnlich verglichen mit dem Testen in reinem PyTorch.

Schlussendlich wird einzig und allein die finale Bewertung des Models zurückgegeben und entschlüsselt.

In [ ]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader[:n_test_batches]:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
# This 'test' function performs the encrypted evaluation. The model weights, the data inputs, the prediction and the target used for scoring are all encrypted!

# However as you can observe, the syntax is very similar to normal PyTorch testing! Nice!

# The only thing we decrypt from the server side is the final score at the end of our 200 items batches to verify predictions were on average good.      
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))


In [ ]:
test(args, model, private_test_loader)

Et voilà! Es wurde vermittelt, wie Ende-zu-Ende sichere Vorhersagen umgesetzt werden können: die Gewichte des vom Server bereit gestellten Models sind weiterhin geheim und auch der Server konnte keinerlei Rückschlüsse über die Eingabe- und Ausgabe-Daten ziehen!

Bezüglich der Performance dauert das Klassifizieren eines Bildes **weniger als 0.1 Sekunden**, ungefähr **33ms** auf dem genutzten Laptop (2,7 GHz Intel Core i7, 16GB RAM).  
Jedoch kann eine schnelle Kommunikation angenommen werden, da alle Helfer auf dem lokalen Gerät angesiedelt sind. Die Performance wird schwanken, weil die einzelnen Helfer unterschiedlich schnell miteinander kommunizieren können.

### PySyft auf GitHub einen Stern geben! 

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos mit Sternen auszuzeichnen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen. 

- [Gib PySyft einen Stern](https://github.com/OpenMined/PySyft)

### Nutze unsere Tutorials auf GitHub!

Wir haben hilfreiche Tutorials erstellt, um ein Verständnis für Federated und Privacy-Preserving Learning zu entwickeln und zu zeigen wie wir die einzelnen Bausteine weiter entwickeln.

- [PySyft Tutorials ansehen](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Mach mit bei Slack! 

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei! 

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Entwickler zu werden! Sie können jederzeit zur PySyft GitHub Issues-Seite gehen und nach "Projects" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "good first issue" gekennzeichnet sind. 

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebase beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups! 

 - [OpenMined's Open Collective Page](https://opencollective.com/openmined)